## Bibliotecas

In [29]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler 
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, mean_squared_error
from keras.models import Sequential
from keras.layers import Dense, LSTM
import matplotlib.pyplot as plt
from pathlib import Path
import os
pd.options.mode.chained_assignment = None

## Ler base

In [30]:
# Caminho da base
base_diretório = Path(os.getcwd())
diretório_princapl = base_diretório.parent
caminho = diretório_princapl / 'Base' / 'Base Ibovespa Tratada.csv'

# Carrega a base
acao = pd.read_csv(caminho)
acao['Data'] = pd.to_datetime(acao['Data'])

# Ordena por data (caso não esteja)
acao = acao.sort_values(by="Data")

acao

,Data,Último,Abertura,Máxima,Mínima,Variacao (%)
0,2015-01-05,47517,48512,48512,47264,-0.0205
1,2015-01-06,48001,47517,48061,47338,0.0102
2,2015-01-07,49463,48006,49882,48006,0.0305
3,2015-01-08,49943,49463,50261,49017,0.0097
4,2015-01-09,48840,49955,49955,48501,-0.0221
...,...,...,...,...,...,...
2577,2025-05-28,138888,139541,139547,138580,-0.0047
2578,2025-05-29,138534,138869,139108,137993,-0.0025
2579,2025-05-30,137027,138546,138637,136726,-0.0109
2580,2025-06-02,136787,137026,138471,136483,-0.0017


In [31]:
# Define ontem e anteontem
ontem = acao['Data'].max()
anteontem = acao['Data'].iloc[-2]

print(f'Ontem: {ontem}\nAnteontem: {anteontem}')

Ontem: 2025-06-03 00:00:00
Anteontem: 2025-06-02 00:00:00


In [32]:
# Filtrando da base de treino até anteontem
acao_treino = acao[acao['Data'] < ontem]
cotacao_treino = acao_treino['Último'].to_numpy().reshape(-1, 1)

In [33]:
# Escala os dados
escalador = MinMaxScaler(feature_range=(0, 1))
dados_escalados = escalador.fit_transform(cotacao_treino)

In [34]:
# Cria dados de entrada para o modelo
treinamento_x = []
treinamento_y = []

for i in range(60, len(dados_escalados)):
    treinamento_x.append(dados_escalados[i-60:i, 0])
    treinamento_y.append(dados_escalados[i, 0])

In [35]:
treinamento_x, treinamento_y = np.array(treinamento_x), np.array(treinamento_y)
treinamento_x = treinamento_x.reshape(treinamento_x.shape[0], treinamento_x.shape[1], 1)

In [36]:
# Define e treina o modelo
modelo = Sequential()
modelo.add(LSTM(100, return_sequences=True, input_shape=(treinamento_x.shape[1], 1)))
modelo.add(LSTM(50, return_sequences=False))
modelo.add(Dense(25))
modelo.add(Dense(1))
modelo.compile(optimizer="adam", loss="mean_squared_error")
modelo.fit(treinamento_x, treinamento_y, batch_size=10, epochs=10)

Epoch 1/10


c:\Users\YUPOPIC\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


253/253 ━━━━━━━━━━━━━━━━━━━━ 9s 25ms/step - loss: 0.0318
Epoch 2/10
253/253 ━━━━━━━━━━━━━━━━━━━━ 6s 24ms/step - loss: 8.3810e-04
Epoch 3/10
253/253 ━━━━━━━━━━━━━━━━━━━━ 6s 23ms/step - loss: 6.9241e-04
Epoch 4/10
253/253 ━━━━━━━━━━━━━━━━━━━━ 6s 24ms/step - loss: 6.2184e-04
Epoch 5/10
253/253 ━━━━━━━━━━━━━━━━━━━━ 6s 22ms/step - loss: 4.8960e-04
Epoch 6/10
253/253 ━━━━━━━━━━━━━━━━━━━━ 5s 20ms/step - loss: 4.7668e-04
Epoch 7/10
253/253 ━━━━━━━━━━━━━━━━━━━━ 6s 23ms/step - loss: 5.5397e-04
Epoch 8/10
253/253 ━━━━━━━━━━━━━━━━━━━━ 6s 22ms/step - loss: 5.8833e-04
Epoch 9/10
253/253 ━━━━━━━━━━━━━━━━━━━━ 8s 31ms/step - loss: 3.9308e-04
Epoch 10/10
253/253 ━━━━━━━━━━━━━━━━━━━━ 8s 31ms/step - loss: 3.6751e-04


In [37]:
# Pega os últimos 60 dias até ANTEONTEM para previsão
ultimos_60 = dados_escalados[-60:]
entrada_predicao = ultimos_60.reshape(1, 60, 1)

In [45]:
# Faz a previsão para ontem
predicao = modelo.predict(entrada_predicao)
predicao = escalador.inverse_transform(predicao)

# Valor real de ontem
valor_real = acao[acao['Data'] == ontem]['Último'].values[0]

# Exibe os resultados
print(f"Previsão para {ontem.date()}: R$ {predicao[0][0]:}")
print(f"Valor real em {ontem.date()}: R$ {valor_real:}")

# Métricas
erro_abs = abs(valor_real - predicao[0][0])
erro_pct = erro_abs / valor_real * 100
print(f"Erro absoluto: R$ {erro_abs:2f}")
print(f"Erro percentual: {erro_pct:.2f}%")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 130ms/step
Previsão para 2025-06-03: R$ 135890.453125
Valor real em 2025-06-03: R$ 136685
Erro absoluto: R$ 794.546875
Erro percentual: 0.58%


In [49]:
predicao[0][0]

np.float32(135890.45)